<a href="https://colab.research.google.com/github/sumaaithal/PySpark_30Days_Challenge/blob/main/pyspark19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [48.6 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,046 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-secu

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [14]:
from pyspark.sql.functions import expr

In [4]:
filepath = "/content/small_zipcode.csv"

In [5]:
df = spark.read.options(header='true', inferSchema='true' ).csv(filepath )

In [6]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- population: integer (nullable = true)



In [7]:
df.show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               NULL|   PR|     30100|
|  2|    704|    NULL|PASEO COSTA DEL SUR|   PR|      NULL|
|  3|    709|    NULL|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               NULL|   TX|      NULL|
+---+-------+--------+-------------------+-----+----------+



In [8]:
df.na.fill(value=0 ).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               NULL|   PR|     30100|
|  2|    704|    NULL|PASEO COSTA DEL SUR|   PR|         0|
|  3|    709|    NULL|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               NULL|   TX|         0|
+---+-------+--------+-------------------+-----+----------+



In [9]:
df.na.fill(value=0 ,subset=['population'] ).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               NULL|   PR|     30100|
|  2|    704|    NULL|PASEO COSTA DEL SUR|   PR|         0|
|  3|    709|    NULL|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               NULL|   TX|         0|
+---+-------+--------+-------------------+-----+----------+



In [10]:
df.na.fill(" ").show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|                   |   PR|     30100|
|  2|    704|        |PASEO COSTA DEL SUR|   PR|      NULL|
|  3|    709|        |       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|                   |   TX|      NULL|
+---+-------+--------+-------------------+-----+----------+



In [11]:
df.na.fill("unknown", subset=["city"]) \
  .na.fill(" ",subset=["type"]).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|            unknown|   PR|     30100|
|  2|    704|        |PASEO COSTA DEL SUR|   PR|      NULL|
|  3|    709|        |       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|            unknown|   TX|      NULL|
+---+-------+--------+-------------------+-----+----------+



In [12]:
df.na.fill({'city':"unknown",'type':" ",'population':0}).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|            unknown|   PR|     30100|
|  2|    704|        |PASEO COSTA DEL SUR|   PR|         0|
|  3|    709|        |       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|            unknown|   TX|         0|
+---+-------+--------+-------------------+-----+----------+



In [15]:
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

columns= ["Product","Amount","Country"]

In [16]:
df = spark.createDataFrame(data=data,schema=columns)
df.show()

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
| Banana|  1000|    USA|
|Carrots|  1500|    USA|
|  Beans|  1600|    USA|
| Orange|  2000|    USA|
| Orange|  2000|    USA|
| Banana|   400|  China|
|Carrots|  1200|  China|
|  Beans|  1500|  China|
| Orange|  4000|  China|
| Banana|  2000| Canada|
|Carrots|  2000| Canada|
|  Beans|  2000| Mexico|
+-------+------+-------+



In [18]:
pivotDF = df.groupBy("product").pivot("country").sum("amount")

pivotDF.show()

+-------+------+-----+------+----+
|product|Canada|China|Mexico| USA|
+-------+------+-----+------+----+
| Orange|  NULL| 4000|  NULL|4000|
|  Beans|  NULL| 1500|  2000|1600|
| Banana|  2000|  400|  NULL|1000|
|Carrots|  2000| 1200|  NULL|1500|
+-------+------+-----+------+----+



In [19]:
countries = ["USA","China","Canada","Mexico"]

pivotDF = df.groupBy("product").pivot("country",countries).sum("amount")
pivotDF.show()

+-------+----+-----+------+------+
|product| USA|China|Canada|Mexico|
+-------+----+-----+------+------+
| Orange|4000| 4000|  NULL|  NULL|
|  Beans|1600| 1500|  NULL|  2000|
| Banana|1000|  400|  2000|  NULL|
|Carrots|1500| 1200|  2000|  NULL|
+-------+----+-----+------+------+



In [20]:
pivotDF = df.groupBy("product","country")\
            .sum("amount")\
            .groupBy("product")\
            .pivot("country")\
            .sum("sum(amount)")

pivotDF.show()

+-------+------+-----+------+----+
|product|Canada|China|Mexico| USA|
+-------+------+-----+------+----+
| Orange|  NULL| 4000|  NULL|4000|
|  Beans|  NULL| 1500|  2000|1600|
| Banana|  2000|  400|  NULL|1000|
|Carrots|  2000| 1200|  NULL|1500|
+-------+------+-----+------+----+



In [23]:
unPivotExpr = "stack(3, 'Canada',Canada,'China',China,'Mexico',Mexico) as (country,total)"

In [26]:
unPivotDF = pivotDF.select("product", expr(unPivotExpr))\
                    .where("total is not null")

unPivotDF.show()

+-------+-------+-----+
|product|country|total|
+-------+-------+-----+
| Orange|  China| 4000|
|  Beans|  China| 1500|
|  Beans| Mexico| 2000|
| Banana| Canada| 2000|
| Banana|  China|  400|
|Carrots| Canada| 2000|
|Carrots|  China| 1200|
+-------+-------+-----+

